<a href="https://colab.research.google.com/github/ev1025/CP1/blob/main/Code_project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 인스타카트 데이터 분석

참고자료 : 케글 인스타카트 장바구니 분석 데이터     
https://www.kaggle.com/datasets/psparks/instacart-market-basket-analysis

# 1. CRISP-DM 기반 분석

## 1) 비즈니스이해

## 2) 데이터 이해

## 3) 데이터 전처리

In [1]:
# 한글 그래프 폰트(런타임재시작, plt.rc('font', family='NanumBarunGothic'))
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20170925-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 10 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/var/cache/fontconfig: cleaning cache directory
/root/.cache/fontconfig: not cleaning non-existent

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
from functools import reduce

In [2]:
df1 = pd.read_csv('/content/drive/MyDrive/인스타카트/aisles.csv')               # 소분류
df2 = pd.read_csv('/content/drive/MyDrive/인스타카트/departments.csv')          # 대분류
df3 = pd.read_csv('/content/drive/MyDrive/인스타카트/order_products__train.csv')
df4 = pd.read_csv('/content/drive/MyDrive/인스타카트/orders.csv')
df5 = pd.read_csv('/content/drive/MyDrive/인스타카트/products.csv')

In [4]:
df3.shape

(1384617, 4)

In [5]:
df1.isnull().sum()

aisle_id    0
aisle       0
dtype: int64

In [6]:
df2.isnull().sum()

department_id    0
department       0
dtype: int64

In [7]:
df3.isnull().sum()

order_id             0
product_id           0
add_to_cart_order    0
reordered            0
dtype: int64

In [8]:
df4.isnull().sum()# 현재 주문 이전 주문과의 텀(데이터 유지)

order_id                       0
user_id                        0
eval_set                       0
order_number                   0
order_dow                      0
order_hour_of_day              0
days_since_prior_order    206209
dtype: int64

In [9]:
df5.isnull().sum() 

product_id       0
product_name     0
aisle_id         0
department_id    0
dtype: int64

In [10]:
# 한 번에 merge 각 컬럼이 달라서 실패
# data = [df3,df1,df2,df5,df6]
# full_data = reduce(lambda x,y : pd.merge(x,y), data)

In [3]:
# 데이터 합치는 함수
def df_merge(t1,t2,t3,t4,t5):
    data = pd.merge(t1,t2)
    data = pd.merge(data,t3)
    data = pd.merge(data,t4)
    data = pd.merge(data,t5)
    return data

In [4]:
full_data = df_merge(df3,df4,df5,df1,df2)
full_data

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,aisle,department
0,1,49302,1,1,112108,train,4,4,10,9.0,Bulgarian Yogurt,120,16,yogurt,dairy eggs
1,816049,49302,7,1,47901,train,14,4,6,16.0,Bulgarian Yogurt,120,16,yogurt,dairy eggs
2,1242203,49302,1,1,2993,train,15,0,7,7.0,Bulgarian Yogurt,120,16,yogurt,dairy eggs
3,1383349,49302,11,1,41425,train,4,3,8,14.0,Bulgarian Yogurt,120,16,yogurt,dairy eggs
4,1787378,49302,8,0,187205,train,5,4,14,30.0,Bulgarian Yogurt,120,16,yogurt,dairy eggs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1384612,3283540,39484,23,1,103639,train,7,0,20,6.0,Sliced Turkey Feast in Gravy Cat Food,41,8,cat food care,pets
1384613,3339757,24892,1,1,157916,train,4,2,6,22.0,Wild Mackerel & Whitefish Cat Food,41,8,cat food care,pets
1384614,3343813,24892,9,0,43638,train,10,2,15,22.0,Wild Mackerel & Whitefish Cat Food,41,8,cat food care,pets
1384615,3343813,27290,10,0,43638,train,10,2,15,22.0,Purrfect Delicacies Wet Cat Food Flaked Oceanf...,41,8,cat food care,pets


In [5]:
# 그래프분석에 사용할 데이터 만들기
df_graph = full_data[['order_id', 'user_id','product_name','aisle','department','order_number','reordered','order_dow','days_since_prior_order']]

In [6]:
# 컬럼명 변경
def df_rename(data):
    data.columns = ['주문번호','유저','제품','소분류','대분류','누적주문','재주문','요일','주문텀']
    return data

In [7]:
df_graph = df_rename(df_graph)
df_graph = df_graph.sort_values(by='주문번호',ignore_index=True)

In [16]:
# CSV로 저장
# df_graph.to_csv('file.csv',index=False)

#### 분석

In [11]:
df_graph.groupby('제품')['주문텀'].max().sort_values(ascending=False).dropna()

제품
#2 Coffee Filters                                        30.0
Organic Lowatf Plain Yogurt                              30.0
Organic Mandarins                                        30.0
Organic Maitake Mushrooms                                30.0
Organic Magnolia and Poppy Scented Glass Cleaner         30.0
                                                         ... 
Aromafier Diffuser, Grey                                  0.0
Medium Picture Hanging Strips                             0.0
Hershey's Chocolate/Strawberry                            0.0
Gallon Size Single Slide Storage Bags                     0.0
Dairy Free 5X Protein Almond Plus Vanilla Almond Milk     0.0
Name: 주문텀, Length: 39123, dtype: float64

In [ ]:
# 대분류 순위
df_graph['대분류'].value_counts(normalize=True)

In [ ]:
count_depart = df_graph['대분류'].value_counts()
count_depart = count_depart.to_frame()
count_depart.reset_index(inplace=True)
count_depart.columns = ['대분류','개수']

In [ ]:
plt.rc('font', size= 15, family='NanumBarunGothic')
plt.rc('xtick',labelsize=12)

plt.figure(figsize =(12,8))
sns.barplot(data=count_depart[:10],x='대분류',y='개수')

plt.show()

In [ ]:
# 대분류 produce의 제품목록
df_graph[df_graph['대분류']=='produce']['제품'].value_counts()

In [ ]:
df_graph.groupby(['요일','대분류'])['유저'].count().loc[0]

In [ ]:
product_count = df_graph['제품'].value_counts()
product_count

In [ ]:
top_5 = product_count[:5].to_frame().reset_index()

In [ ]:
top_5.rename(columns={'index':'제품명'},inplace=True)

In [ ]:
plt.figure(figsize=(10,7))
sns.barplot(data = top_5, x='제품명',y='제품')
plt.show()

In [ ]:
# 재주문이 많은 제품 top20
df_graph.groupby('제품')['재주문'].sum().sort_values(ascending=False).head(20)

In [ ]:
df_graph['소분류'].value_counts(normalize=True)

In [ ]:
df_days = df_graph.groupby('요일')['유저'].count()
df_days = df_days.to_frame()
df_days.reset_index(inplace=True)
df_days

In [ ]:
plt.figure(figsize=(8,6))
sns.barplot(data = df_days, x=df_days['요일'],y= df_days['유저'] )

In [ ]:
day_depart = df_graph.groupby('요일')['대분류'].value_counts()
day_depart = day_depart.to_frame()
day_depart

In [ ]:
day_depart.loc[0]

In [ ]:
day_aisles = df_graph.groupby('요일')['소분류'].value_counts()
day_aisles = day_aisles.to_frame()
day_aisles

In [ ]:
day_aisles.loc[1]

## 4) 모델링

k-means

In [8]:
df_graph

(1384617, 9)

In [9]:
df_graph.loc[0]['주문번호']
df_graph.loc[5]['주문번호']

1

In [42]:
df_graph.values[0][0] == df_graph.values[1][0]

True

In [111]:
df_graph.head(31)

,주문번호,유저,제품,소분류,대분류,누적주문,재주문,요일,주문텀
0,1,112108,Bulgarian Yogurt,yogurt,dairy eggs,4,1,4,9.0
1,1,112108,Lightly Smoked Sardines in Olive Oil,canned meat seafood,canned goods,4,1,4,9.0
2,1,112108,Organic Celery Hearts,fresh vegetables,produce,4,0,4,9.0
3,1,112108,Organic Whole String Cheese,packaged cheese,dairy eggs,4,1,4,9.0
4,1,112108,Organic Hass Avocado,fresh fruits,produce,4,0,4,9.0
5,1,112108,Bag of Organic Bananas,fresh fruits,produce,4,0,4,9.0
6,1,112108,Organic 4% Milk Fat Whole Milk Cottage Cheese,other creams cheeses,dairy eggs,4,1,4,9.0
7,1,112108,Cucumber Kirby,fresh vegetables,produce,4,0,4,9.0
8,36,79431,Asparagus,fresh vegetables,produce,23,1,6,30.0
9,36,79431,Grated Pecorino Romano Cheese,specialty cheeses,dairy eggs,23,0,6,30.0


In [112]:
for i in range(30):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [126]:
df_graph.tail(7)

,주문번호,유저,제품,소분류,대분류,누적주문,재주문,요일,주문텀
1384610,3421063,169679,Twice Baked Potatoes,prepared meals,deli,30,1,0,4.0
1384611,3421063,169679,No Salt Added Gluten-Free Blue Chips Made with...,chips pretzels,snacks,30,1,0,4.0
1384612,3421063,169679,Organic Half & Half,cream,dairy eggs,30,1,0,4.0
1384613,3421063,169679,Natural Artesian Water,water seltzer sparkling water,beverages,30,1,0,4.0
1384614,3421070,139822,Creamy Peanut Butter,spreads,pantry,15,1,6,8.0
1384615,3421070,139822,Organic Unsweetened Almond Milk,soy lactosefree,dairy eggs,15,1,6,8.0
1384616,3421070,139822,Broccoli Florettes,packaged produce,produce,15,1,6,8.0


In [127]:
record[0]

['Bulgarian Yogurt']

In [128]:
abc = []
for i in range(len(df_array)):
    abc.append([df_array[j][2] for j in range(len(df_array)) if df_array[i][0]==df_array[j][0]])

KeyboardInterrupt: ignored

In [223]:
df_graph.head(33)

,주문번호,유저,제품,소분류,대분류,누적주문,재주문,요일,주문텀
0,1,112108,Bulgarian Yogurt,yogurt,dairy eggs,4,1,4,9.0
1,1,112108,Lightly Smoked Sardines in Olive Oil,canned meat seafood,canned goods,4,1,4,9.0
2,1,112108,Organic Celery Hearts,fresh vegetables,produce,4,0,4,9.0
3,1,112108,Organic Whole String Cheese,packaged cheese,dairy eggs,4,1,4,9.0
4,1,112108,Organic Hass Avocado,fresh fruits,produce,4,0,4,9.0
5,1,112108,Bag of Organic Bananas,fresh fruits,produce,4,0,4,9.0
6,1,112108,Organic 4% Milk Fat Whole Milk Cottage Cheese,other creams cheeses,dairy eggs,4,1,4,9.0
7,1,112108,Cucumber Kirby,fresh vegetables,produce,4,0,4,9.0
8,36,79431,Asparagus,fresh vegetables,produce,23,1,6,30.0
9,36,79431,Grated Pecorino Romano Cheese,specialty cheeses,dairy eggs,23,0,6,30.0


In [222]:
record[3]

['Roasted Turkey',
 'Organic Cucumber',
 'Organic Grape Tomatoes',
 'Organic Blueberries',
 'Organic Raspberries',
 'Organic Pomegranate Kernels',
 'Organic Whole Strawberries']

In [201]:
# 주문번호 별 주문 리스트 나누기(한 번 주문 할 때 어떤 물건을 같이 사는지 확인하기 위해서)

# 연관규칙분석을 위해 요소 비교를 하는 과정에서 시간이 오래걸려서 리스트 대신 array를 사용하였다.
# 하나의 iterator 요소간 비교에서 zip 함수를 사용 할 수 있었다.
# 마지막 열은 record에 append 되지 않는 점을 확인하여 if문을 하나 더 추가하여 마지막 열 추가

df_array = df_graph.values # 전체 데이터를 array화 하여 계산속도 증가

record = [] # 주문번호 별 주문내역리스트
record2 =[] # 한 주문에 포함 된 내역

for p1, p2 in zip(df_array, df_array[1:]): # 0과1 1과2 ... 비교하도록 for문 

    if p1[0]==p2[0]:              # 주문번호가 같으면 record2에 추가
        record2.append(p1[2])

        if p1[2]==df_array[-2][2]: # 전체데이터의 마지막열에 도달하면 마지막 주문내역 record에 추가
            record2.append(p2[2])
            record.append(record2)
  
    else:
        record2.append(p1[2])    # 주문번호가 다르면 해당 주문의 마지막 물건이다. 주문번호 마지막 목록을 record2에 추가하고
        record.append(record2)   # 해당 주문을 그룹화한 record2를 record에 추가하고
        record2 = []             # record2를 초기화

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

df_model = df_graph[['누적주문','주문텀']].dropna()
scaler = MinMaxScaler()
sc_data = scaler.fit_transform(df_model)

In [ ]:
# model = KMeans(n_clusters=10, random_state=12)
# model.fit(sc_data)

## 5) 평가

## 6) 배포